In [1]:
import time
import numpy as np
import pinocchio as pin
from robot_descriptions.loaders.pinocchio import load_robot_description
from utils.meshcat_viewer_wrapper import MeshcatVisualizer


In [2]:
robot = load_robot_description('ur5_description')
viz = MeshcatVisualizer(robot)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7002/static/


Lets visualize the neutral position of the robot in the next cell and then we will print out the frames contained in the robot model.

In [3]:
q = pin.neutral(robot.model)
viz.display(q)

In [4]:
for i in range(robot.model.nframes):
    frame = robot.model.frames[i]
    print(f"Frame {i}: {frame.name}")

Frame 0: universe
Frame 1: root_joint
Frame 2: world
Frame 3: world_joint
Frame 4: base_link
Frame 5: base_link-base_fixed_joint
Frame 6: base
Frame 7: shoulder_pan_joint
Frame 8: shoulder_link
Frame 9: shoulder_lift_joint
Frame 10: upper_arm_link
Frame 11: elbow_joint
Frame 12: forearm_link
Frame 13: wrist_1_joint
Frame 14: wrist_1_link
Frame 15: wrist_2_joint
Frame 16: wrist_2_link
Frame 17: wrist_3_joint
Frame 18: wrist_3_link
Frame 19: ee_fixed_joint
Frame 20: ee_link
Frame 21: wrist_3_link-tool0_fixed_joint
Frame 22: tool0


The following prints out the joint frames, these are typically the actuated joints of the robot. In a pinocchio model there is also always a Joint 0 which corresponds to the world frame which is named universe.

In [5]:
for i in range(robot.model.njoints):
    joint = robot.model.joints[i]
    joint_name = robot.model.names[i]
    print(f"Joint {i}: {joint_name}")

Joint 0: universe
Joint 1: shoulder_pan_joint
Joint 2: shoulder_lift_joint
Joint 3: elbow_joint
Joint 4: wrist_1_joint
Joint 5: wrist_2_joint
Joint 6: wrist_3_joint


Now lets print out the position of each joint frame.

In [6]:
for name, oMi in zip(robot.model.names, robot.data.oMi):
    print(("{:<24} : {: .2f} {: .2f} {: .2f}"
          .format( name, *oMi.translation.T.flat )))

universe                 :  0.00  0.00  0.00
shoulder_pan_joint       :  0.00  0.00  0.00
shoulder_lift_joint      :  0.00  0.00  0.00
elbow_joint              :  0.00  0.00  0.00
wrist_1_joint            :  0.00  0.00  0.00
wrist_2_joint            :  0.00  0.00  0.00
wrist_3_joint            :  0.00  0.00  0.00


As you can see all frames are zero which is not correct. That is because we are using robot.data to print them. In order for the data to be updated in robot.data, we have to call the forward kinematics. Also any other info like velocities, accelerations, forces etc are contained in this data structure. For more info on this take a look at the Data section of the cheatsheet https://github.com/stack-of-tasks/pinocchio/blob/master/doc/pinocchio_cheat_sheet.pdf

Now lets update the frames with the forward kinematics and reprint the frame locations.

In [7]:
pin.framesForwardKinematics(robot.model, robot.data, pin.neutral(robot.model))
for name, oMi in zip(robot.model.names, robot.data.oMi):
    print(("{:<24} : {: .2f} {: .2f} {: .2f}"
          .format( name, *oMi.translation.T.flat )))
print(robot.data.v[6])

universe                 :  0.00  0.00  0.00
shoulder_pan_joint       :  0.00  0.00  0.09
shoulder_lift_joint      :  0.00  0.14  0.09
elbow_joint              :  0.42  0.02  0.09
wrist_1_joint            :  0.82  0.02  0.09
wrist_2_joint            :  0.82  0.11  0.09
wrist_3_joint            :  0.82  0.11 -0.01
  v = 0 0 0
  w = 0 0 0



To get specific frames we can use the following:

In [8]:
IDX_BASE = robot.model.getFrameId('base_link')
IDX_TOOL = robot.model.getFrameId('tool0')

oMtool = robot.data.oMf[IDX_TOOL]
oMbase = robot.data.oMf[IDX_BASE]

print("Tool placement:\n",oMtool)
print("Base placement:\n",oMbase)

Tool placement:
   R =
          -1 -9.79328e-12  4.79541e-23
           0  4.89664e-12            1
-9.79328e-12            1 -4.89664e-12
  p =   0.81725   0.19145 -0.005491

Base placement:
   R =
1 0 0
0 1 0
0 0 1
  p = 0 0 0



Now lets visualize a random velocity trajectory. We can integrate the velocities into positions with the pin.integrate function.

In [33]:
vq = np.random.rand(robot.model.nv)*4 - 2
DT = 1e-3
n_steps = 1000
for t in range(n_steps):
    q = pin.integrate(robot.model,q,vq*DT)
    if t%10==0:
        viz.display(q)
    time.sleep(1/n_steps)

In the next section we will use inverse velocity kinematics to find the joint configuration that corresponds to an end effector pose. First lets visualize the goal frame. This section of the tutorial will correspond do the numerical inverse kinematics algorithm from section 6.2.2 of *Modern Robotics*.

In [34]:
oMgoal = pin.SE3(pin.utils.rotate('x',np.pi), np.array([0.2, .4, .7]))
viz.visualize_frame("oMgoal", oMgoal)

You can find the Jacobian of the tool frame with the following command.

In [35]:
Jtool = pin.computeFrameJacobian(robot.model, robot.data, q, IDX_TOOL)
Jtool.shape

(6, 6)

As we saw in the lecture, using the inverse jacobian we can map twists in a robot frame to joint velocities.
$$\dot{q} = J^{\dagger}_{tool}(\theta) \mathcal{V}_{tool}$$
We can rewrite this to only contain the increments dq and dx in the following way and then define dx as the error between the tool and goal frames.
$$dq = J^{\dagger}_{tool}(\theta) dx$$
$$dx = \log({}^0 T_{tool}^{-1} {}^0 T_{goal})$$

In [36]:
q = pin.neutral(robot.model)
viz.display(q)
n_steps = 10000
for i in range(n_steps+1):

    # Run the algorithms that outputs values in robot.data
    pin.framesForwardKinematics(robot.model,robot.data,q)
    pin.computeJointJacobians(robot.model,robot.data,q)

    # Placement from world frame o to frame f oMtool  
    oMtool = robot.data.oMf[IDX_TOOL]

    # 6D error between the two frame
    tool_nu = pin.log(oMtool.inverse()*oMgoal).vector

    # Get corresponding jacobian
    tool_Jtool = pin.computeFrameJacobian(robot.model, robot.data, q, IDX_TOOL, pin.LOCAL)

    # Control law by least square
    vq = np.linalg.pinv(tool_Jtool)@tool_nu

    q = pin.integrate(robot.model,q, vq * DT)
    if i%10==0:
        viz.display(q)
    time.sleep(1/n_steps)


Now lets visualize the tool frame to see is it corresponds to the goal. If you cant see a difference, then they should be overlapping. You can delete them one after another to confirm this with the last two cells.

In [37]:
oMtool = robot.data.oMf[IDX_TOOL]
viz.visualize_frame("oMtool", oMtool)

In [38]:
viz.delete("oMgoal")

In [39]:
viz.delete("oMtool")